In [7]:
import nltk
print("Works!")

Works!


In [10]:
#Loading the cha files directly into this pycharm booklet

import os 
import re 
import pandas as pd


CHA_DIR = "/Users/kennedycameron/Downloads/Hoff/"

cha_files = [f for f in os.listdir(CHA_DIR) if f.endswith(".cha")]

utterances = []

for filename in cha_files:
    age = None
    filepath = os.path.join(CHA_DIR, filename)

    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()

            if line.startswith("@ID:") and "CHI" in line:
                age_match = re.search(r"\|\d;(\d{2})\.", line)
                if age_match:
                    age = age_match.group(1)

            # only grab the lines when the children are speaking
            if line.startswith("*CHI:"):
                utterance = re.sub(r"\*[A-Z]+:\s*", "", line)  
                utterance = utterance.split("%")[0]

                # remove timestamps and symbols like & and +
                utterance = re.sub(r"[^\w\s\?\!\.\,áéíóúüñÁÉÍÓÚÜÑ]", "", utterance)

                # & double spaces
                utterance = re.sub(r"\s{2,}", " ", utterance).strip()

                utterances.append({"file": filename,"age": age, "utterance": utterance})
 
df = pd.DataFrame(utterances)
print("Total child utterances:", len(df))
print(df.head())


Total child utterances: 12256
         file age          utterance
0  028(1).cha  06     better . 0_857
1  028(1).cha  06  uhhuh ! 3286_3853
2  028(1).cha  06  uhhuh . 5837_6844
3  028(1).cha  06   mmhm . 7790_8446
4  028(1).cha  06              xxx .


In [ ]:
#Cleaning the children babble as much as possible
print("Before cleaning:", len(df))
df["utterance"] = df["utterance"].str.replace(r"\d+_\d+", "", regex=True)

filler_words = {"uhhuh","mmhm","yeah","um","uh"}
df = df[~df["utterance"].str.lower().isin(filler_words)]
df = df[~df["utterance"].str.contains(r"\bxxx\b|\byyy\b", regex=True)]
df = df[df["utterance"].str.contains(r"[A-Za-záéíóúüñÁÉÍÓÚÜÑ]")]
# & the lines without letters

print("After cleaning:", len(df))
print(df.head(10))



Before cleaning: 12256
After cleaning: 9695
          file age            utterance
0   028(1).cha  06            better . 
1   028(1).cha  06             uhhuh ! 
2   028(1).cha  06             uhhuh . 
3   028(1).cha  06              mmhm . 
5   028(1).cha  06     I want animals .
6   028(1).cha  06            bridge . 
9   028(1).cha  06     I need horsey . 
10  028(1).cha  06              uhhuh !
12  028(1).cha  06     I like animals .
13  028(1).cha  06  this one is farm . 
